In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
from hyperopt import hp,fmin,tpe,STATUS_OK
import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns: #faktoryzacja cech
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x)=='None' else x.split(' ')[0])
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x) )
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else str(x).split('cm')[0].replace(' ', ''))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [8]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}
run_model(xgb.XGBRegressor(**xgb_params), feats) #teraz wszystko jest liczbą, nie ma None

[17:00:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

Hyperopt

In [13]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK }

  #space
xgb_reg_params = {
      'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
      'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
      'subsample': hp.quniform('subsample', 0.05, 1, 0.05),
      'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
      'objective': 'reg:squarederror',
      'n_estimators': 100,
      'seed': 0,
  }

  ## run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.3, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.4}
Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.35000000000000003}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 

{'colsample_bytree': 0.8,
 'learning_rate': 0,
 'max_depth': 9,
 'subsample': 0.9500000000000001}

#Upload na Github

In [0]:
!git config --global user.email "interes@o2.pl"
!git config --global user.name "Rafal L"

In [16]:
!git add "day5.ipynb"
!git commit -m "grid search po wybranym przedziale hiperparametrów modelu"
!git push -u origin master

fatal: pathspec 'day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   Matrix2-day4.ipynb
	modified:   day2_visualization.ipynb
	modified:   day3-simple-model.ipynb

no changes added to commit
Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
